In [71]:
import numpy as np
import scipy as sp
from scipy.special import loggamma as lg
from scipy.integrate import quad, quad_vec, fixed_quad
import matplotlib.pyplot as plt
#from quadpy import quad

In [35]:
def rothwell_lead(v, z):

    lead = 0.5*np.log(np.pi)-lg(v+0.5)-v*np.log(2*z)-z
    
    lead[np.isinf(lead)] = -z + 0.5 * np.log(0.5*np.pi / z)

    return lead

def inner_integral_rothwell(u, v, z):

    n = 8
    v_mhalf = v - 0.5
    neg2v_m1 = -2. * v - 1.
    beta = (2. * n ) / (2. * v + 1.)

    uB = u**beta
    inv_u = -1. / u

    first = beta * np.exp(-uB) * (2 * z + uB)**v_mhalf * u**(n-1)
    second = np.exp(inv_u)
    if second > 0:
        second *= u**neg2v_m1
        if np.isinf(second):
            second = np.exp(inv_u+neg2v_m1*np.log(u))
        second *= (2. * z * u + 1)**v_mhalf

    return first + second

def compute_log_integral(v, z):
    
    integral = np.zeros(np.shape(v))
    
    for i, v_tmp in enumerate(v):
        
        integral[i] = quad(inner_integral_rothwell,0,1,args=(v_tmp, z))[0]
        
    return np.log(integral)

def rothwell(v, z):

	lead = rothwell_lead(v, z)
	log_integral = compute_log_integral(v, z)

	return lead + log_integral

In [11]:
def asymptotic_large_v(v, z):

	return lg(v) - np.log(2) + v * (np.log(2)-np.log(z))

In [85]:
def asymptotic_large_z(v, z):

    log_z = np.log(z)
    base = 0.5 * (np.log(np.pi) - np.log(2) - log_z) - z

    max_terms = 50
    v_squared_4 = v * v * 4
    a_k_z_k = 1
    series_sum = 1

    for k in range(1, max_terms+1):
        a_k_z_k *= (v_squared_4-(2 * k -1)**2) / (k * z * 8)
        series_sum += a_k_z_k
        #if np.abs(a_k_z_k) < 1e-8:
        #    print("breeaaaak")
        #    break

    return base + np.log(series_sum)

In [86]:
def rothwell_log_z_boundary(v):
    
    rothwell_max_log_z_over_v = 300

    return rothwell_max_log_z_over_v / (v-0.5)-np.log(2)

def method_indices(v, z):

    rothwell_max_v = 50
    rothwell_max_z = 100000
    rothwell_max_log_z_over_v = 300

    rothwell_1 = v < rothwell_max_v
    rothwell_2 = np.log(z) < rothwell_log_z_boundary(v)

    i_rothwell = np.logical_and(rothwell_1, rothwell_2)

    i_asymp_v = np.logical_and(v > z, ~i_rothwell)
    
    i_asymp_z = np.logical_and(np.logical_and(v > 10, ~i_rothwell),
                               ~i_asymp_v)
    

    return i_rothwell, i_asymp_z, i_asymp_v

In [111]:
def log_bessel_k(v, z):
    
    v = np.abs(v)
    res = np.zeros(np.shape(v))
    methods = [rothwell, asymptotic_large_z, asymptotic_large_v]

    indeces = method_indices(v, z)
    
    for method, index in zip(methods, indeces):

        res[index] = method(v[index], z)

    return res, indeces

# TESTS

In [112]:
v_to_test = np.array([0,3.15e-7, 2.62e-6, 1.3e-5, 9.2e-5, 0.0026,
                      0.0843, 0.17345, 1, 1.63, 7.42, 42.42424, 86.5,
                      113.8, 148.7565, 180.6, 246.3, 300.5, 513.6,
                      712.456, 714.456, 1235.6,8656, 15330.75,
                      37634.2 ,85323])

z_to_test = np.array([1.48e-7, 3.6e-6, 7.248e-5, 4.32e-4, 8.7e-3, 0.04523, 0.17532,
                      1, 3, 11.32465, 105.6, 1038.4, 4236, 11457.6, 62384, 105321.6,
                      158742.3, 196754,  1.98e6])

In [113]:
def recursion_test(v, z):
    
    first = log_bessel_k(v-2, z)[0]
    display(np.log(v-1))
    second = np.log(2)+np.log(v-1)-np.log(z)+log_bessel_k(v-1, z)[0]
    
    rhs = np.logaddexp(first, second)
    lhs = log_bessel_k(v, z)[0]
    
    return lhs/rhs

In [114]:
recursion_test(v_to_test, z_to_test[10])

/home/jacob/anaconda3/envs/pulsar_timing/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/home/jacob/anaconda3/envs/pulsar_timing/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,        -inf, -0.46203546,
        1.85941812,  3.72386622,  4.44851638,  4.72561634,  4.99556565,
        5.19073216,  5.50248195,  5.70211442,  6.23949581,  6.56731357,
        6.57012077,  7.11850231,  9.06589247,  9.63755066, 10.53564192,
       11.35418761])

/home/jacob/anaconda3/envs/pulsar_timing/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/home/jacob/anaconda3/envs/pulsar_timing/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """


array([            nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
        1.00000000e+00, -5.37586399e+50,  1.00000000e+00,  1.00000000e+00,
        1.00000027e+00,  1.00704478e+00,  9.61865061e-01,  9.97874424e-01,
        9.99651642e-01,  9.99860074e-01,  9.99983779e-01,  9.99995169e-01,
        9.99995217e-01,  9.99999312e-01,  9.99999999e-01,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00])